In [4]:
##Preparing data as per https://doi.org/10.1186/s12920-018-0460-9 
##Predicting drug response of tumors from integrated genomic profiles by deep neural networks
##Yu-Chiao Chiu, Hung-I Harry Chen, Tinghe Zhang, Songyao Zhang, Aparna Gorthi, Li-Ju Wang, Yufei Huang

In [469]:
import pandas as pd
import numpy as np
import h5py

In [478]:
#TCGA_exp = pd.read_csv('counts_gene.tsv', delimiter='\t')  --> TCGA exp downloaded from GDC Portal is not used , it contains 58.000 genes, most of them from intron, 
#while those downloaded from https://amp.pharm.mssm.edu/archs4/download.html  contains a selection of 25.150 genes

In [480]:
#print(TCGA_exp.shape)
#TCGA_exp.columns[-1]

(58037, 11285)


'gene_id'

In [15]:
#TCGA_exp.loc[TCGA_exp['3DFF72D2-F292-497E-ACE3-6FAA9C884205'] == 439935,'gene_id']

50432    ENSG00000268895.5
Name: gene_id, dtype: object

In [20]:
import mygene
mg = mygene.MyGeneInfo()
mg.getgene('ENSG00000268895', 'name,symbol,refseq.rna') 

{'_id': '503538',
 '_score': 13.213065,
 'name': 'A1BG antisense RNA 1',
 'refseq': {'rna': 'NR_015380.2'},
 'symbol': 'A1BG-AS1'}

In [98]:
mg.getgene(150465, 'name,symbol,refseq.rna')

{'_id': '150465',
 '_score': 14.277951,
 'name': 'tubulin tyrosine ligase',
 'refseq': {'rna': ['NM_153712.5', 'XM_005263599.3', 'XM_011510665.2']},
 'symbol': 'TTL'}

In [473]:
#Importing TCGA RNA expression data downloaded from  https://amp.pharm.mssm.edu/archs4/download.html
filename = "../tcga_matrix.h5"

h5 = h5py.File(filename,'r')
for k in h5.keys():
    print(k)


meta = h5['meta']  
dat = h5['data']  
info = h5['info']  



#Showing which type of data it contains    
for s in meta.keys():
    print('meta')
    print(s)
    print(meta[s].shape)
    print(meta[s].value)
    
for s in dat.keys():
    print('data')
    print(s)
    print(dat[s].shape)
    print(dat[s].value)    
    
for s in info.keys():
    print(s)
    print(info[s].shape)
    print(info[s].value)    

#h5.close()

meta
data
info
meta
auc
(11284,)
[b'3116748351' b'5069849036' b'4882401623' ... b'5742234142' b'6773612787'
 b'7613399024']
meta
bigwig_file
(11284,)
[b'3DFF72D2-F292-497E-ACE3-6FAA9C884205.bw'
 b'B1E54366-42B9-463C-8615-B34D52BD14DC.bw'
 b'473713F7-EB41-4F20-A37F-ACD209E3CB75.bw' ...
 b'76987B28-B56B-4A1F-B77C-1E08B8EFEF90.bw'
 b'33737781-8638-4FA2-AD4C-E888BB9343D8.bw'
 b'0BB05CA7-C6FF-42A1-919C-D801F471CBBD.bw']
meta
cancertype
(11284,)
[b'Liver Hepatocellular Carcinoma' b'Prostate Adenocarcinoma'
 b'Rectum Adenocarcinoma' ... b'Brain Lower Grade Glioma'
 b'Acute Myeloid Leukemia' b'Thyroid Carcinoma']
meta
gdc_annotations
(11284,)
[b'NULL' b'NULL' b'NULL' ... b'NULL'
 b'list(category = "Alternate sample pipeline", status = "Approved", entity_id = "fff35c80-88cd-4923-80c1-0273ba5bed0f", classification = "Notification", updated_datetime = "2016-05-01T15:00:21.638875-05:00", created_datetime = "2012-11-13T12:37:22-05:00", entity_submitter_id = "TCGA-AB-2881", notes = "Biospecimens fro

 [  40426   40053      48 ...  793314  481256  313210]]
author
(1,)
[b'Alexander Lachmann']
contact
(1,)
[b'alexander.lachmann@mssm.edu']
countsource
(1,)
[b'recount2: https://jhubiostatistics.shinyapps.io/recount/']
creationdate
(1,)
[b'10/9/2017']
lab
(1,)
[b"Ma'ayan Lab, Icahn School of Medicine"]


In [472]:
#if not exists it saves a file 'log2expression.npy' with the log2 transformation of RNA expression 
import os.path
def f(x):
    #return log2(TPM+0.001)
    return np.log2(x +0.001 ) 

l = None
if not os.path.exists('../log2expression.npy'):
    #log2(TPM+0.001) expression values computed per gene
    l = dat['expression'].value
    l = np.vectorize(f)(l)
    np.save('log2expression.npy', l)
else: 
    l = np.load('../log2expression.npy')

from scipy import stats
stats.describe(l)

DescribeResult(nobs=11284, minmax=(array([ 4.90693868,  4.39238612, -9.96578428, ..., 14.11845455,
       12.98815228, 11.60733078]), array([25.74434345, 23.45781508, 21.78148379, ..., 24.41667745,
       22.1506869 , 21.65159744])), mean=array([14.51133518, 14.65828565,  7.81866386, ..., 19.87816157,
       18.0625713 , 17.54948585]), variance=array([ 5.12112323,  3.17381026, 58.31272768, ...,  1.31941998,
        1.13436237,  0.97464454]), skewness=array([ 1.34799737,  1.15626794, -0.97460423, ..., -0.09400296,
       -0.34315363, -0.48017077]), kurtosis=array([4.18004613, 4.2873423 , 0.95231487, ..., 0.81602377, 0.62193682,
       0.94856403]))

In [476]:
#associate gene names and case_id with RNA expression 
def f(x):
    return x.decode("utf-8")
genes = np.vectorize(f)(meta['genes'].value)
cases_id  = np.vectorize(f)(meta['gdc_cases.case_id'].value)
df = pd.DataFrame(l, columns = genes, index= cases_id)


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0004d251-3f70-4395-b175-c94c2f5b1b81,20.472352,18.746931,18.272466,19.290201,15.257498,-9.965784,9.299210,5.643885,13.595724,-9.965784,...,13.189979,14.272849,11.805341,13.000000,15.517669,9.278452,15.699383,18.206232,14.866072,15.448536
000d566c-96c7-4f1c-b36e-fa2222467983,14.818083,14.545749,7.584970,21.170981,17.082887,13.392988,10.586841,3.322072,17.862692,-9.965784,...,14.685789,14.368779,13.460072,15.752668,16.611414,12.744834,16.470659,19.945104,17.252194,16.725619
0011a67b-1ba9-4a32-a6b8-7850759a38cf,12.077484,13.246592,17.234948,20.949892,14.822969,6.700454,10.472692,7.321937,16.977783,7.044405,...,17.196554,17.947871,15.637785,17.493347,16.861135,8.839207,17.636320,20.331971,18.412549,17.949416
001887aa-36d0-463f-8bca-dec7043b4f2e,24.457885,22.684448,19.907039,23.860669,19.778088,7.918869,13.952559,1.000721,16.081317,-9.965784,...,14.631916,14.412041,12.749870,15.258603,16.945592,12.561527,18.267911,16.341345,17.340346,17.422426
001944e5-af34-4061-9c09-bb9ea346f6fd,11.676398,12.256504,8.076821,19.457004,14.529736,17.318905,8.209458,-9.965784,17.581149,-9.965784,...,17.754470,19.272135,13.757911,15.446534,17.096242,12.186734,17.651276,20.299427,17.006589,17.689343


In [483]:
print(df.shape)
TCGA_exp_matrix = df.transpose()

(11284, 25150)


In [82]:
#GDSC (not used) -> sanger1018_brainarray_ensemblgene_rma.txt, other gen_drug ML predictor: FORESEE https://academic.oup.com/bioinformatics/advance-article/doi/10.1093/bioinformatics/btz145/5367831 
#CCLE exp -> v21.data.gex_avg_log2.txt, v21.meta.gex_features.txt is downloaded from ftp://caftpd.nci.nih.gov/pub/OCG-DCC/CTD2/Broad/CTRPv2.0_2015_ctd2_ExpandedDataset/ 
CCLE_read_exp = pd.read_csv('CTRPv2.1_2016_pub_NatChemBiol_12_109/v21.data.gex_avg_log2.txt', delimiter='\t')

In [83]:
CCLE_read_exp.columns

Index(['idx_gene_feature', 'master_ccl_id', 'mrna_expression_avg_log2'], dtype='object')

In [135]:
CCLE_meta = pd.read_csv('CTRPv2.1_2016_pub_NatChemBiol_12_109/v21.meta.gex_features.txt', delimiter='\t')
CCLE_meta_cells = pd.read_csv('CTRPv2.1_2016_pub_NatChemBiol_12_109/v21.meta.per_cell_line.txt', delimiter='\t')
print(CCLE_meta.columns)
#master_ccl_id   ccl_name
print(CCLE_meta_cells.columns)

Index(['idx_gene_feature', 'affy_probeset_id', 'entrez_gene_id',
       'gene_primary_name', 'gene_symbol'],
      dtype='object')
Index(['master_ccl_id', 'ccl_name', 'ccle_primary_site', 'ccle_primary_hist',
       'ccle_hist_subtype_1'],
      dtype='object')


In [102]:
CCLE_meta.shape

(18543, 5)

In [205]:
CCLE_gen_exp = pd.merge(CCLE_meta[['idx_gene_feature','gene_primary_name']], CCLE_read_exp, on=['idx_gene_feature'])
CCLE_gen_exp = pd.merge(CCLE_meta_cells[['master_ccl_id','ccl_name']], CCLE_gen_exp, on=['master_ccl_id'])

CCLE_gen_exp.drop(columns = ['idx_gene_feature', 'master_ccl_id'], inplace=True)


In [206]:
#making compatible CCLE identifiers between sources
CCLE_gen_exp['CELL_LINE_NAME']= CCLE_gen_exp['ccl_name'].str.replace('-','')

In [207]:
CCLE_gen_exp.drop(columns = ['ccl_name'], inplace=True)

In [208]:
CCLE_gen_exp= CCLE_gen_exp.set_index(['CELL_LINE_NAME','gene_primary_name'])
CCLE_gen_exp.shape

(15260889, 1)

In [209]:
#There are duplicated combinations of genes and cell lines, due to multiple probes that need to be averaged 
#CCLE_gen_exp[(CCLE_gen_exp['master_ccl_id']==1 ) & (CCLE_gen_exp['gene_primary_name']=='TTL' )]

CCLE = CCLE_gen_exp.groupby(['CELL_LINE_NAME','gene_primary_name']).mean()

In [210]:
CCLE.shape


(15259243, 1)

In [211]:
CCLE_matrix = CCLE.unstack(level=0)

In [212]:
CCLE_matrix

mrna_expression_avg_log2                                   \
CELL_LINE_NAME                       22RV1    2313287       253J     253JBV   
gene_primary_name                                                             
A1BG                              4.980354   4.460359   4.783156   4.875861   
A1BG-AS1                          4.434383   4.496125   4.156233   4.115867   
A1CF                              7.531097   6.034458   3.698593   3.576494   
A2M                               4.732491   3.804899   4.503684   4.604683   
A2M-AS1                           5.487521   4.381788   4.358112   4.278722   
A2ML1                             3.374869   3.440269   3.552498   3.665464   
A4GALT                            3.704898   4.359996   4.114879   4.030298   
A4GNT                             3.965432   3.990351   4.148009   4.152585   
AAAS                              9.142425   8.057531   7.679034   7.670979   
AACS                             10.232410   8.807101   8.416268   9.025579   
AACSP1                            5.824946   4.715717   4.768379   5.027295   
AADAC                             3.650851   3.985758   3.789363   3.663323   
AADACL2                           3.593321   3.964709   3.785144   3.973737   
AADAT                             7.174712   4.880775   4.997732   4.989466   
AAED1                             7.073925   8.000179   8.502624   9.252570   
AAGAB                             7.831102   8.279278   9.205090   8.993416   
AAK1                              4.865253   4.531757   5.135262   4.730520   
AAMDC                             5.908342   6.865500   6.237460   5.968700   
AAMP                              8.971352   8.555001   8.380440   8.698034   
AANAT                             3.850252   3.469509   4.061703   3.911838   
AAR2                              8.415848   8.477459   8.027367   8.322819   
AARS                             10.758420  11.497330  10.096570  10.167480   
AARS2                             7.100141   6.806494   6.493925   6.837733   
AARSD1                            4.682500   4.649159   4.585120   4.722699   
AASDH                             8.234263   7.776226   7.415976   7.905038   
AASDHPPT                          9.972673   9.934601   9.751606   9.993719   
AASS                              5.088142   4.914488   5.040493   5.142298   
AATF                             10.383670  10.641440  10.810290  11.053560   
AATK                              5.948981   6.535075   5.661927   5.539595   
ABAT                              5.412974   4.544920   5.593541   4.777535   
...                                    ...        ...        ...        ...   
ZSCAN22                           6.642765   5.951252   6.077843   5.699935   
ZSCAN23                           3.457134   3.410110   3.906051   3.756745   
ZSCAN25                           6.665363   6.346621   6.072987   6.481707   
ZSCAN26                           6.507847   6.680639   6.354317   6.561463   
ZSCAN29                           7.335548   8.706523   7.257616   7.500642   
ZSCAN30                           6.901050   6.410693   6.508031   6.737665   
ZSCAN31                           5.127202   7.054289   5.321745   5.104675   
ZSCAN32                           6.827015   7.181184   7.231475   6.787155   
ZSCAN4                            3.553442   3.637095   3.915194   3.809703   
ZSCAN5A                           5.911231   5.412088   5.104043   5.245742   
ZSCAN9                            7.083867   6.961542   6.719567   6.451054   
ZSWIM1                            6.726215   7.362105   7.125602   6.921156   
ZSWIM2                            3.425063   3.523792   3.554232   3.428999   
ZSWIM3                            5.576534   5.878665   4.949871   5.355911   
ZSWIM5                            6.162788   5.973207   4.929678   4.489325   
ZSWIM6                            7.642530   8.495682  10.322000   9.351223   
ZSWIM7                            7.755569   7.278704   7.830481   7

In [273]:
#IC50 for CCLE is obtained from supplement material in https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4967469/#mmc5
IC50 = pd.read_excel('mmc5.xlsx', sheet_name='TableS4A-IC50s',  skiprows = range(1, 5), header=1, index_col=1)


In [274]:

IC50.drop(columns= 'Unnamed: 0', inplace = True)
IC50.index = IC50.index.str.replace('-','')


In [275]:
IC50 = IC50.transpose()
IC50

,ALLPO,AMO1,COLO668,CORL95,DG75,ECGI10,ES5,GT3TKB,JJN3,KINGS1,...,MM1S,EW12,H2810,NCIH128,CORL32,ECC12,KPNRTBM1,CP67MEL,NCIH378,NCIH250
TL-2-105,1.412465,3.363134,3.398414,3.312450,3.650059,5.001921,1.448888,4.490765,2.208866,5.381778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TAK-715,2.113150,3.051047,3.103017,6.096484,3.107754,4.003636,1.929536,3.736883,3.011577,6.112720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CP466722,-0.756692,1.718606,3.067645,2.950619,2.175176,5.448208,1.367814,3.789823,1.366097,6.000134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BMS-345541,1.386112,2.191936,1.228204,2.848541,1.587807,3.371173,1.536514,4.036876,2.032780,4.328236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genentech Cpd 10,-1.398149,1.541583,0.313308,3.740211,2.438582,5.000838,-0.783992,2.562712,1.361857,2.415087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSK429286A,3.681923,3.544126,3.572447,5.216102,5.301156,5.671351,3.937856,4.799862,3.797401,6.282322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ruxolitinib,3.081088,2.989808,3.731025,3.719220,4.153156,4.331940,3.574216,3.651463,3.421299,4.987484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SB-715992,-3.230122,-3.888980,-2.275653,-0.931930,-2.527416,0.385862,-4.086624,-4.855325,-3.716937,-1.357139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZSTK474,-1.691070,0.275043,0.577291,2.301070,2.322550,4.308773,-1.944276,3.063025,0.262038,3.227872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KIN001-102,-0.722249,1.626300,0.940925,2.685076,2.099412,5.001921,-0.463507,4.492776,1.595150,4.407665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
#cell lines without IC50 for specific drugs are inputed with kNN (k = 5) using KNN from VIM package of R (other jupyter to be uploaded)


In [279]:
print(IC50.shape) #They downloaded drug response data of 990 CCLE cell lines to 265 anti-cancer
#drugs measured by the half maximal inhibitory concentration (IC50) from the GDSC Project.

(265, 990)


In [178]:
#MAF - Fields:entity_id, case_id, Hugo_Symbol
#four types of nonsynonymous mutations, including missense and nonsense mutations, frameshift insertions and deletions


In [185]:
# CCLE MAF downloaded from https://data.broadinstitute.org/ccle/CCLE_DepMap_18q3_maf_20180718.txt
#Note: TODO to use the newest version in 2019 from https://depmap.org/portal/download/all/?release=DepMap+Public+19Q1&file=DepMap-2019q1-celllines_v2.csv
CCLE_maf_read = pd.read_csv('CCLE_DepMap_18q3_maf_20180718.txt', delimiter='\t')

/home/auri/anaconda3/envs/gen/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [186]:
CCLE_maf_read

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,COSMIChsCnt,ExAC_AF,WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Broad_ID
0,DVL1,1855,37,1,1277461,1277461,+,Silent,SNP,C,...,0,NaN,87:39,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
1,AL590822.1,0,37,1,2144416,2144416,+,Missense_Mutation,SNP,G,...,0,NaN,23:43,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
2,PLCH2,9651,37,1,2435359,2435359,+,Splice_Site,SNP,A,...,0,NaN,9:25,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
3,UBE4B,10277,37,1,10177641,10177641,+,Missense_Mutation,SNP,G,...,0,NaN,13:95,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
4,SRM,6723,37,1,11119363,11119363,+,Silent,SNP,G,...,0,NaN,113:57,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
5,BAI2,576,37,1,32205607,32205607,+,Missense_Mutation,SNP,C,...,0,NaN,101:84,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
6,S100PBP,64766,37,1,33321630,33321630,+,Silent,SNP,G,...,0,0.000008,74:109,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
7,ZSCAN20,7579,37,1,33958951,33958951,+,Missense_Mutation,SNP,C,...,0,NaN,110:210,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
8,SZT2,23334,37,1,43909122,43909122,+,Nonsense_Mutation,SNP,G,...,0,NaN,86:144,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270
9,MOB3C,148932,37,1,47078936,47078936,+,Missense_Mutation,SNP,G,...,0,0.000025,131:58,NaN,NaN,NaN,NaN,NaN,NaN,ACH-001270


In [184]:
#CCLE_ID only match CELL_LINE_NAME if "_" is removed from CELL_LINE_NAME and then mapped to  CCLE_ID removing _*
CCLE_annot = pd.read_csv('Cell_lines_annotations_20181226.txt', delimiter='\t')
CCLE_annot.columns

Index(['CCLE_ID', 'depMapID', 'Name', 'Pathology', 'Site_Primary',
       'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'Histology',
       'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Gender',
       'Life_Stage', 'Age', 'Race', 'Geo_Loc', 'inferred_ethnicity',
       'Site_Of_Finding', 'Disease', 'Annotation_Source',
       'Original.Source.of.Cell.Line', 'Characteristics', 'Growth.Medium',
       'Supplements', 'Freezing.Medium', 'Doubling.Time.from.Vendor',
       'Doubling.Time.Calculated.hrs', 'type', 'type_refined',
       'PATHOLOGIST_ANNOTATION', 'mutRate', 'tcga_code'],
      dtype='object')

In [362]:
CCLE_maf = pd.merge(CCLE_annot[['CCLE_ID','depMapID']],CCLE_maf_read, left_on=['depMapID'], right_on=['Broad_ID'] )

In [363]:
CCLE_maf.drop(columns=['Broad_ID','depMapID'], inplace = True)

In [364]:
CCLE_maf['CELL_LINE_NAME'] = CCLE_maf['CCLE_ID'].str.replace(r'_.*','')

In [366]:
#select only 4 types of mutations: four types of nonsynonymous mutations, including missense and nonsense mutations, 
#frameshift insertions and deletions --> only those are tagged as mutated genes in the binary matrices
print(CCLE_maf.columns)
set(CCLE_maf['Variant_Classification'].values)
print(CCLE_maf.groupby(['Variant_Classification','Variant_Type']).count())

Index(['CCLE_ID', 'Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'Tumor_Sample_Barcode', 'cDNA_Change', 'Codon_Change', 'Protein_Change',
       'isDeleterious', 'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot',
       'COSMIChsCnt', 'ExAC_AF', 'WES_AC', 'SangerWES_AC',
       'SangerRecalibWES_AC', 'RNAseq_AC', 'HC_AC', 'RD_AC', 'WGS_AC',
       'CELL_LINE_NAME'],
      dtype='object')
                                       CCLE_ID  Hugo_Symbol  Entrez_Gene_Id  \
Variant_Classification   Variant_Type                                         
3'UTR                    DEL                50           50              50   
                         INS                12           12              12   
5'Flank                  DEL                64      

In [367]:
CCLE_maf = CCLE_maf[['Variant_Classification', 'Hugo_Symbol', 'CELL_LINE_NAME']]

In [368]:
mutated = ['Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins', 'Missense_Mutation','Nonsense_Mutation']
CCLE_maf.loc[:,'Mutated']   = 0
CCLE_maf.loc[CCLE_maf['Variant_Classification'].isin(mutated),'Mutated'] = 1

In [392]:
CCLE_maf.Mutated.value_counts()



1    738013
0    419289
Name: Mutated, dtype: int64

In [2]:
CCLE_maf.drop(columns=['Variant_Classification'], inplace = True)

In [378]:
CCLE_maf_matrix = CCLE_maf.groupby([ 'CELL_LINE_NAME','Hugo_Symbol']).count()

In [379]:
CCLE_maf_matrix = CCLE_maf_matrix.unstack(level=0)

In [406]:
CCLE_maf_matrix.shape

(19278, 1414)

In [401]:
CCLE_maf_matrix.fillna(value = 0) #all combinations of gene and cell line not reported in the MAF file is assumed to be wildtype 

Mutated                                                        \
CELL_LINE_NAME    201T 22RV1 2313287 253J 253JBV 42MGBA 451LU 5637  59M 639V   
Hugo_Symbol                                                                    
A1BG               0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
A1CF               0.0   1.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
A2M                0.0   0.0     1.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
A2ML1              0.0   1.0     1.0  0.0    0.0    0.0   0.0  0.0  0.0  1.0   
A3GALT2            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
A4GALT             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
A4GNT              0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AAAS               0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AACS               0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AADAC              0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AADACL2            1.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AADACL3            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AADACL4            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AADAT              0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AAED1              0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AAGAB              0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AAK1               0.0   1.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AAMDC              0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AAMP               0.0   1.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AANAT              0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AAR2               0.0   0.0     1.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AARD               0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AARS               0.0   1.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AARS2              0.0   1.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AARSD1             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AASDH              1.0   0.0     0.0  0.0    0.0    1.0   0.0  0.0  0.0  0.0   
AASDHPPT           0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AASS               0.0   1.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  1.0   
AATF               1.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
AATK               0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  1.0   
...                ...   ...     ...  ...    ...    ...   ...  ...  ...  ...   
ZSCAN29            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSCAN30            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSCAN31            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  1.0  0.0   
ZSCAN32            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSCAN4             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSCAN5A            0.0   1.0     0.0  0.0    0.0    0.0   0.0  1.0  0.0  0.0   
ZSCAN5B            0.0   0.0     0.0  0.0    0.0    0.0   0.0  1.0  0.0  0.0   
ZSCAN5C            0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSCAN9             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSWIM1             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSWIM2             0.0   0.0     1.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSWIM3             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  1.0   
ZSWIM4             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSWIM5             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSWIM6             0.0   1.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSWIM7             0.0   0.0     0.0  0.0    0.0    0.0   0.0  0.0  0.0  0.0   
ZSWIM8         

In [446]:
import os
import glob

#TCGA MAF data is obtained from GDC Portal using a manifest to download this query MAF files + open: 
#https://portal.gdc.cancer.gov/repository?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.access%22%2C%22value%22%3A%5B%22open%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.data_format%22%2C%22value%22%3A%5B%22MAF%22%5D%7D%7D%5D%7D
#long process 
if not os.path.exists('maf_TCGA.csv'):
    paths = glob.glob('maf_TCGA/**/*.somatic.maf.gz' , recursive=True)

    df = pd.DataFrame()
    i = 0
    for maf in paths:
        i+=1
        print(i)
        d = pd.read_csv(maf, compression='gzip', delimiter='\t', skiprows= 5)
        if d.columns[0] == 'Hugo_Symbol' :
            df = df.append(d)
        else:
            print(d.columns[0])
    df.to_csv('maf_TCGA.csv')
            
else:    
    TCGA_maf_read = pd.read_csv('maf_TCGA.csv', usecols=['Variant_Classification', 'Variant_Type','Hugo_Symbol', 'case_id'])


    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


/home/auri/anaconda3/envs/gen/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (87,98,118) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


20


/home/auri/anaconda3/envs/gen/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,20,28,87,98,118) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


21
22
23


...

85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121


/home/auri/anaconda3/envs/gen/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,20,28,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


122
123


/home/auri/anaconda3/envs/gen/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,20,28,88,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


124
125
126
127
128
129
130
131


In [449]:
#select only 4 types of mutations: four types of nonsynonymous mutations, including missense and nonsense mutations, 
#frameshift insertions and deletions --> only those are tagged as mutated genes in the binary matrices
print(TCGA_maf_read.columns)
set(TCGA_maf_read['Variant_Classification'].values)
print(TCGA_maf_read.groupby(['Variant_Classification','Variant_Type']).count())

Index(['Hugo_Symbol', 'Variant_Classification', 'Variant_Type', 'case_id'], dtype='object')
                                     Hugo_Symbol  case_id
Variant_Classification Variant_Type                      
3'Flank                DEL                  6202     6202
                       INS                  1779     1779
                       SNP                 78791    78791
3'UTR                  DEL                 73092    73092
                       INS                 21134    21134
                       SNP                737635   737635
5'Flank                DEL                  2183     2183
                       INS                   732      732
                       SNP                 62431    62431
5'UTR                  DEL                  8361     8361
                       INS                  2933     2933
                       SNP                205110   205110
Frame_Shift_Del        DEL                166957   166957
Frame_Shift_Ins        INS            

In [450]:

mutated = ['Frame_Shift_Del','Frame_Shift_Ins','In_Frame_Del','In_Frame_Ins', 'Missense_Mutation','Nonsense_Mutation']
TCGA_maf_read.loc[:,'Mutated']   = 0
TCGA_maf_read.loc[TCGA_maf_read['Variant_Classification'].isin(mutated),'Mutated'] = 1

In [451]:
TCGA_maf_read.Mutated.value_counts()

1    6525124
0    4391042
Name: Mutated, dtype: int64

In [452]:
TCGA_maf_read.drop(columns=['Variant_Classification','Variant_Type'], inplace = True)

In [454]:
TCGA_maf_matrix = TCGA_maf_read.groupby([ 'case_id','Hugo_Symbol']).count()

In [455]:
TCGA_maf_matrix = TCGA_maf_matrix.unstack(level=0)

In [456]:
TCGA_maf_matrix.fillna(value = 0) #all combinations of gene and cell line not reported in the MAF file are assumed to be wildtype 

Mutated  \
case_id         0004d251-3f70-4395-b175-c94c2f5b1b81   
Hugo_Symbol                                            
A1BG                                             0.0   
A1CF                                             0.0   
A2M                                              0.0   
A2ML1                                            0.0   
A2MP1                                            0.0   
A3GALT2                                          0.0   
A4GALT                                           0.0   
A4GNT                                            0.0   
AAAS                                             0.0   
AACS                                             0.0   
AACSP1                                           0.0   
AADAC                                            0.0   
AADACL2                                          0.0   
AADACL2-AS1                                      0.0   
AADACL3                                          0.0   
AADACL4                                          0.0   
AADAT                                            0.0   
AAED1                                            0.0   
AAGAB                                            0.0   
AAK1                                             0.0   
AAMDC                                            0.0   
AAMP                                             0.0   
AANAT                                            0.0   
AAR2                                             0.0   
AARD                                             0.0   
AARS                                             0.0   
AARS2                                            0.0   
AARSD1                                           0.0   
AASDH                                            0.0   
AASDHPPT                                         0.0   
...                                              ...   
ZSCAN5A                                          0.0   
ZSCAN5B                                          0.0   
ZSCAN5C                                          0.0   
ZSCAN9                                           0.0   
ZSWIM1                                           0.0   
ZSWIM2                                           0.0   
ZSWIM3                                           0.0   
ZSWIM4                                           0.0   
ZSWIM5                                           0.0   
ZSWIM6                                           0.0   
ZSWIM7                                           0.0   
ZSWIM8                                           0.0   
ZUFSP                                            0.0   
ZW10                                             0.0   
ZWILCH                                           0.0   
ZWINT                                            0.0   
ZXDA                                             0.0   
ZXDB                                             0.0   
ZXDC                                             0.0   
ZYG11A                                           0.0   
ZYG11B                                           0.0   
ZYX                                              0.0   
ZZEF1                                            0.0   
ZZZ3                                             0.0   
bP-21201H5.1                                     0.0   
bP-2189O9.2                                      0.0   
hsa-mir-1253                                     0.0   
hsa-mir-6511b-1                                  0.0   
pk                                               0.0   
snoU13                                           0.0   

                                                      \
case_id         000d566c-96c7-4f1c-b36e-fa2222467983   
Hugo_Symbol                                            
A1BG                                             0.0   
A1CF                                             0.0   
A2M                                              0.0   
A2ML1                                            0.0   
A2MP1                                            0.0   
A3GALT2     

In [458]:
#Genes with no mutations in CCLE and TCGA samples were eliminated.
genes_mut_TCGA_CCLE = set(TCGA_maf_matrix.index.values)and set(CCLE_maf_matrix.index.values)
print(len(genes_mut_TCGA_CCLE))

19278


In [467]:
TCGA_maf_matrix_r = TCGA_maf_matrix.loc[list(genes_mut_TCGA_CCLE),:]
TCGA_maf_matrix_r.shape

(19278, 10189)

In [466]:
CCLE_maf_matrix_r = CCLE_maf_matrix.loc[list(genes_mut_TCGA_CCLE),:]
CCLE_maf_matrix_r.shape

(19278, 1414)

In [487]:
#Summary of data
print(len(set(CCLE_maf['CELL_LINE_NAME'].values)))
print(len(IC50.columns)) #cell lines
print(len(set(CCLE_matrix.columns)))
print(len(set(CCLE_maf['Hugo_Symbol'].values)))
print("N Cell lines with MAF + expression + IC50 information")
#Counts of cell lines with MAF + expression + IC50 information
print(len( set(IC50.columns) and set(CCLE_maf['CELL_LINE_NAME'].values) and set(CCLE_matrix.columns.values)))
#Couts of genes included in expression and MAF files
print("N genes with MAF and expression in cell lines")
print(len(set(CCLE_maf['Hugo_Symbol'].values) and set(CCLE_matrix.index.values)))
print('N of drugs tested on the cell lines')
print(len(IC50.index))
#Counts of tumors with TCGA MAF and gene expression
print("N tumors with TCGA MAF and gene expression")
tumors_TCGA = set(TCGA_maf_matrix.columns.values)and set(TCGA_exp_matrix.columns.values)
print(len(tumors_TCGA))


1414
990
823
19278
N Cell lines with MAF + expression + IC50 information
823
N genes with MAF and expression in cell lines
18541
N of drugs tested on the cell lines
265
N tumors with TCGA MAF and gene expression
10340


In [496]:
TCGA_exp_matrix.head() 
TCGA_exp_matrix.to_csv('E_TCGA.csv')

In [497]:
CCLE_matrix.head() 
CCLE_matrix.to_csv('E_CCLE.csv')

In [498]:
TCGA_maf_matrix.head()
TCGA_maf_matrix.to_csv('M_TCGA.csv')

In [499]:
CCLE_maf_matrix.head()
CCLE_maf_matrix.to_csv('M_CCLE.csv')

In [3]:
IC50.head()
IC50.to_csv('IC50_CCLE.csv')